In [1]:
import numpy as np
import pandas as pd
import json
import pickle
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.style.use('default')

In [3]:
import sys, os
sys.path.append(os.path.join(os.path.abspath(''), '..', 'shared_libs'))
import data_transform

In [4]:
df = pd.read_csv('data/data_target_cleared.csv')
df.head()

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,for sale,NaN,multi-family,803 Passmore St,2 Baths,"{'atAGlanceFacts': [{'factValue': '1950', 'fac...",yes,Philadelphia,"[{'rating': ['3/10', '2/10', '3/10'], 'data': ...","1,350 sqft",19111,3 Beds,PA,2.0,NaN,NaN,PAPH853202,195000.0
1,for sale,NaN,lot/land,3609 Summit Ave,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Greensboro,"[{'rating': ['5/10', '5/10', '4/10'], 'data': ...",NaN,27405,NaN,NC,NaN,NaN,NaN,930640,199000.0
2,Active,NaN,Land,4011 Valley Vista Dr,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Lowell,"[{'rating': ['7', '6'], 'data': {'Distance': [...",0,49331,NaN,MI,NaN,NaN,NaN,19058608,189900.0
3,For sale,NaN,Single Family,118 S 17th St,Bathrooms: 2,"{'atAGlanceFacts': [{'factValue': '1909', 'fac...",NaN,Allentown,"[{'rating': ['2/10', '2/10', '2/10'], 'data': ...","Total interior livable area: 1,617 sqft",18104,3 bd,PA,2.0,628531,NaN,NaN,189000.0
4,for sale,NaN,single-family home,6820 Quincy St,4 Baths,"{'atAGlanceFacts': [{'factValue': '1925', 'fac...",yes,Philadelphia,"[{'rating': ['3/10', '1/10'], 'data': {'Distan...","5,013 sqft",19119,6 Beds,PA,3.0,NaN,NaN,PAPH851112,579000.0


In [5]:
df = data_transform.clear_data_base_line(
    df, 
    '../shared_libs/data/default_values.pkl',
    can_drop_rows=True, 
    force_rebuild_cached_data=True
)

In [6]:
cities_dict = data_transform.get_cities_dict('../shared_libs/data/cities_dict.pkl', force_read=True)
address_dict = data_transform.get_addresses_dict('../shared_libs/data/address_dict.pkl', force_read=True)
address_by_zip_dict = data_transform.get_address_by_zipcode_dict('../shared_libs/data/address_by_zip_dict.pkl', force_read=True)
cities_clusters_dict = data_transform.get_citiess_clusters_dict('../shared_libs/data/cities_clusters_dict.pkl', force_read=True)

In [7]:
df = data_transform.fix_incorrect_states_and_cities(
    df, 
    '../shared_libs/data/default_values.pkl', 
    cities_dict, 
    can_drop_rows=True
)

In [8]:
df = data_transform.add_city_features(
    df,
    '../shared_libs/data/default_values.pkl', 
    cities_dict, 
    address_dict,
    address_by_zip_dict,
    cities_clusters_dict,
    force_rebuild_cached_data=True
)

In [9]:
df = data_transform.add_population_features(
    df, 
    '../shared_libs/data/default_values.pkl', 
    '../shared_libs/data/uscities.csv', 
    can_drop_rows=True,
    force_rebuild_cached_data=True
)

In [10]:
df = data_transform.encode_state_and_city(
    df, 
    '../shared_libs/data/default_values.pkl',
    can_drop_rows=True, 
    force_rebuild_cached_data=True
)

In [11]:
df = data_transform.final_tune_pca_and_scale(
    df, 
    '../shared_libs/data/default_values.pkl', 
    force_rebuild_cached_data=True
)

In [12]:
#####################################################################################################################
#####################################################################################################################
#####################################################################################################################
#####################################################################################################################

In [13]:
df_base = df.copy()

In [14]:
y = df_base['target']
X = df_base.drop(['target'], axis=1)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

### Подбор наиболее эффективной модели

In [16]:
model = LinearRegression()

model.fit(X_train, y_train)

y_pred_test = model.predict(X_test)

print('MAPE:', mean_absolute_percentage_error(y_test, y_pred_test)*100)
print('RMSE:', mean_squared_error(y_test, y_pred_test)**0.5)

MAPE: 89.68589530547789
RMSE: 446618.7727140654


In [17]:
with open('../shared_libs/data/models/model_lr.pkl', 'wb') as f:
    pickle.dump(model, f)

In [17]:
model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)

print('Train MAPE:', mean_absolute_percentage_error(y_train, y_pred_train)*100)
print('Train RMSE:', mean_squared_error(y_train, y_pred_train)**0.5)
print('Test MAPE:', mean_absolute_percentage_error(y_test, y_pred_test)*100)
print('Test RMSE:', mean_squared_error(y_test, y_pred_test)**0.5)

Train MAPE: 49.86187035206407
Train RMSE: 246560.114617964
Test MAPE: 54.8209839740759
Test RMSE: 325391.10030879045


Модель на базе **RandomForestRegressor** показывает улучшение результатов по MAPE с 60% до 54,8%.

Попробуем подобрать более эфективную модель.

In [19]:
with open('../shared_libs/data/models/model_rfr.pkl', 'wb') as f:
    pickle.dump(model, f)

### Модель на базе AdaBoostRegressor

"За кадром" для всех нижеследующих моделей был произведен подбор гиперпараметров. Далее приводятся модели с оптимальными гиперпараметрами.

In [20]:
model = AdaBoostRegressor(
    DecisionTreeRegressor(
        min_samples_leaf=4,
        max_depth=27,
        random_state=42
    ),
    random_state=42,
    learning_rate=0.0001,
    n_estimators=100
)

model.fit(X_train, y_train)

y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)

print('Train MAPE:', mean_absolute_percentage_error(y_train, y_pred_train)*100)
print('Train RMSE:', mean_squared_error(y_train, y_pred_train)**0.5)
print('Test MAPE:', mean_absolute_percentage_error(y_test, y_pred_test)*100)
print('Test RMSE:', mean_squared_error(y_test, y_pred_test)**0.5)

Train MAPE: 15.160181354365495
Train RMSE: 166241.58117034868
Test MAPE: 30.176375171879172
Test RMSE: 310733.2767712826


In [21]:
with open('../shared_libs/data/models/model_abr.pkl', 'wb') as f:
    pickle.dump(model, f)

### Модель на базе GradientBoostingRegressor

In [28]:
model = GradientBoostingRegressor(
    loss='absolute_error',
    n_estimators=250,
    learning_rate=0.1,
    min_samples_leaf=2,
    max_depth=13,
    random_state=42
)

model.fit(X_train, y_train)

y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)

print('Train MAPE:', mean_absolute_percentage_error(y_train, y_pred_train)*100)
print('Train RMSE:', mean_squared_error(y_train, y_pred_train)**0.5)
print('Test MAPE:', mean_absolute_percentage_error(y_test, y_pred_test)*100)
print('Test RMSE:', mean_squared_error(y_test, y_pred_test)**0.5)

Train MAPE: 17.727541098514322
Train RMSE: 267869.86105599086
Test MAPE: 28.746294339274865
Test RMSE: 320209.30800889316


In [29]:
with open('../shared_libs/data/models/model_gbr.pkl', 'wb') as f:
    pickle.dump(model, f)

### Модель на базе XGBRegressor

In [67]:
model = XGBRegressor(
    objective='reg:squarederror', 
    eta=0.1,
    max_depth=8,
    n_estimators=2000,
    n_jobs=-1,
    random_state=42
)

model.fit(X_train, y_train)

y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)

print('Train MAPE:', mean_absolute_percentage_error(y_train, y_pred_train)*100)
print('Train RMSE:', mean_squared_error(y_train, y_pred_train)**0.5)
print('Test MAPE:', mean_absolute_percentage_error(y_test, y_pred_test)*100)
print('Test RMSE:', mean_squared_error(y_test, y_pred_test)**0.5)

Train MAPE: 25.71185432704086
Train RMSE: 137380.46631759795
Test MAPE: 40.1932764877937
Test RMSE: 288928.26464646735


In [68]:
with open('../shared_libs/data/models/model_xgbr.pkl', 'wb') as f:
    pickle.dump(model, f)

### Модель на базе CatBoostRegressor

In [35]:
model = CatBoostRegressor(
    iterations=1750,
    learning_rate=0.1,
    depth=14,
    l2_leaf_reg=2,
    verbose=False,
    loss_function='RMSE',
    random_state=42
)

model.fit(X_train, y_train)

y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)

print('Train MAPE:', mean_absolute_percentage_error(y_train, y_pred_train)*100)
print('Train RMSE:', mean_squared_error(y_train, y_pred_train)**0.5)
print('Test MAPE:', mean_absolute_percentage_error(y_test, y_pred_test)*100)
print('Test RMSE:', mean_squared_error(y_test, y_pred_test)**0.5)

Train MAPE: 20.00258477265686
Train RMSE: 119150.08166927722
Test MAPE: 40.016761323703136
Test RMSE: 296764.81853716867


In [36]:
with open('../shared_libs/data/models/model_cbr.pkl', 'wb') as f:
    pickle.dump(model, f)

### Выводы

По совокупным результатам двух метрик (MAPE и RMSE) наилучшие результаты показала модель на базе **AdaBoostRegressor**.

Эта модель и будет использоваться в сервисе предсказания цены недвижимости.